In [1]:
import sys 
sys.path.append('..')
import alpenglow.files as af
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.external.tifffile as tiff
from skimage import transform
import numpy as np
import shutil as sh
import os
import boto3
from scipy import optimize
import alpenglow.stitching
from imp import reload
reload(alpenglow.stitching)
from alpenglow.stitching import stitch, find_shift, apply_shift

In [2]:
from skimage.feature import register_translation
from skimage import img_as_float
from skimage import exposure
from skimage.io import imread_collection, ImageCollection
from skimage.external.tifffile import imread

In [3]:
s3 = boto3.resource('s3')

bucket = s3.Bucket('alpenglowoptics')

ll = list(bucket.objects.all())

In [9]:
for x in range(1,129):
    for strip in [7,8,9,10,11,12]:
        fname = "im_%06d_%06d.tif" % (strip, x)
        af.download_s3('76AO/Strips/' + fname, '../data/' + fname)

In [10]:
images_7 = imread_collection("../data/im_000007_0*")
images_8 = imread_collection("../data/im_000008_0*")
images_9 = imread_collection("../data/im_000009_0*")
images_10 = imread_collection("../data/im_000010_0*")
images_11 = imread_collection("../data/im_000011_0*")
images_12 = imread_collection("../data/im_000012_0*")


In [11]:
def stitch_zstack(images_1, images_2, current_stack):
    """
    
    """
    sub_images_1 = []
    sub_images_2 = []
    ps = []
    for x in range(10,100,10):
        p = int(np.percentile(np.arange(len(images_1)), x))
        sub_images_1_p = images_1[p]
        sub_images_2_p = images_2[p]
        sub_images_1.append(sub_images_1_p)
        sub_images_2.append(sub_images_2_p)
        ps.append(p)
    
    shift = []
    for x in range(len(sub_images_1)):
        shift.append(find_shift(sub_images_1[x], sub_images_2[x]))
    shift = np.array(shift)
    coef = []
    coef = np.polyfit(ps, shift[:,0], 1)
    
    shift_last = int(np.round(np.polyval(coef, len(images_1))))
    rows = images_1[0].shape[0]+images_2[0].shape[0]
    overlap = (images_2[0].shape[0]//2)+shift_last
    chop_index = int(rows - overlap)

    for z_level in range(len(images_1)):
        shift_zero = int(np.round(np.polyval(coef, z_level)))
        shift_one = int(np.round(np.mean(shift[:,1])))
        registered = apply_shift(images_1[z_level], images_2[z_level], [shift_zero, shift_one])
        tiff.imsave("mosaic_%06d_%06d.tif"%(current_stack, z_level+1), registered[:chop_index]) 
        

In [14]:
stitch_zstack(images_7, images_8, 8)

In [12]:
mosaic_8 = ImageCollection("mosaic_000008*", plugin="tifffile")

In [13]:
mosaic_8[0]

IndexError: There are only 0 images in the collection

In [ ]:
test_55 = imread("mosaic_000009_000055.tif")

In [ ]:
test_55.shape

In [ ]:
plt.matshow(test_55[:, 5000:15000])

In [ ]:
pwd

In [ ]:
rm mosaic_000008*